In [40]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
import gmaps
import os

# Import API key
from api_keys import g_key

# Incorporate citipy to determine city based on latitude and longitude
from citipy import citipy

# Import weather data from Part 1
csv_file = os.path.join('..', 'WeatherPy', 'output_data', 'cities.csv')
vacation_df = pd.read_csv(csv_file)
vacation_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Nizwá,0,OM,1604282371,32,22.93,57.53,59.31,2.35
1,1,Dzhebariki-Khaya,0,RU,1604282372,83,62.22,135.80,-4.76,3.27
2,2,Miles City,1,US,1604282373,39,46.41,-105.84,50.00,5.82
3,3,Chai Badan,74,TH,1604282374,77,15.20,101.13,79.36,10.31
4,4,Pak Phanang,40,TH,1604282376,94,8.35,100.20,80.60,4.70
...,...,...,...,...,...,...,...,...,...,...
586,586,East Wenatchee Bench,1,US,1604283122,53,47.43,-120.28,50.00,5.82
587,587,San Vicente,0,AR,1604283123,66,-26.62,-54.13,59.59,4.41
588,588,Ostrovnoy,100,RU,1604283125,92,68.05,39.51,39.00,23.42
589,589,College,20,US,1604283073,77,64.86,-147.80,-0.99,1.63


### Create a heat map that displays the humidity for every city from the part I of the homework.

In [43]:
gmaps.configure(api_key=g_key)

locations = vacation_df[["Lat", "Lng"]]
humidity = vacation_df["Humidity"]

# Create a humidity Heatmap layer using gmaps
fig = gmaps.figure(center = (0, 0), zoom_level=1.75)
heat_layer = gmaps.heatmap_layer(
    locations,
    weights=humidity, 
    dissipating=False,
    max_intensity=100,
    point_radius = 1
)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create sub-dataset containing only cities with weather in my ideal specifications.

In [42]:
### Vacation Weather Criteria: temp 60-80, humidity 10%-50%, wind <10mph
vacation_df.drop(vacation_df[vacation_df["Max Temp"] > 80].index, inplace = True)
vacation_df.drop(vacation_df[vacation_df["Max Temp"] < 60].index, inplace = True)
vacation_df.drop(vacation_df[vacation_df["Humidity"] > 50].index, inplace = True)
vacation_df.drop(vacation_df[vacation_df["Humidity"] < 10].index, inplace = True)
vacation_df.drop(vacation_df[vacation_df["Wind Speed"] < 10].index, inplace = True)
vacation_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
39,39,Umm Kaddadah,18,SD,1604282423,25,13.60,26.69,78.37,22.28
65,65,Bilma,0,NE,1604282457,28,18.69,12.92,68.38,10.18
105,105,Rawson,100,AR,1604282345,25,-43.30,-65.10,75.07,24.87
178,178,Mount Gambier,15,AU,1604282602,36,-37.83,140.77,78.80,13.87
230,230,Kidal,4,ML,1604282668,23,18.44,1.41,73.20,11.30
288,288,Port Lincoln,43,AU,1604282742,39,-34.73,135.87,77.25,13.47
338,338,Pirapozinho,0,BR,1604282734,46,-22.28,-51.50,71.60,11.41
381,381,Jiménez,100,MX,1604282863,35,28.33,-105.40,66.18,12.39
423,423,Bariloche,0,AR,1604282761,42,-41.15,-71.31,64.40,11.41
529,529,Kutum,0,SD,1604283050,34,14.20,24.67,69.85,16.82


### Create map of hotels in my vacation destinations

In [45]:
# Add blank column HOTEL NAME to Vacation DataFrame, so we can populate it with hotels
vacation_df['Hotel Name'] = ""
vacation_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
39,39,Umm Kaddadah,18,SD,1604282423,25,13.60,26.69,78.37,22.28,
65,65,Bilma,0,NE,1604282457,28,18.69,12.92,68.38,10.18,
105,105,Rawson,100,AR,1604282345,25,-43.30,-65.10,75.07,24.87,
178,178,Mount Gambier,15,AU,1604282602,36,-37.83,140.77,78.80,13.87,
230,230,Kidal,4,ML,1604282668,23,18.44,1.41,73.20,11.30,
288,288,Port Lincoln,43,AU,1604282742,39,-34.73,135.87,77.25,13.47,
338,338,Pirapozinho,0,BR,1604282734,46,-22.28,-51.50,71.60,11.41,
381,381,Jiménez,100,MX,1604282863,35,28.33,-105.40,66.18,12.39,
423,423,Bariloche,0,AR,1604282761,42,-41.15,-71.31,64.40,11.41,
529,529,Kutum,0,SD,1604283050,34,14.20,24.67,69.85,16.82,


In [48]:
# Loop through list of cities, and find hotels within 5000meters of the Lat-Lon for each city
for index, row in vacation_df.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }
        lat = row['Lat']
        lng = row['Lng']
        params['location'] = f"{lat}, {lng}"
        hotel_data = requests.get(base_url, params=params).json()
        vacation_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
    # If no hotel within 5000meters, store NaN as Hotel Name    
    except IndexError: 
        vacation_df.loc[index, "Hotel Name"] = "NaN"
vacation_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
39,39,Umm Kaddadah,18,SD,1604282423,25,13.60,26.69,78.37,22.28,NaN
65,65,Bilma,0,NE,1604282457,28,18.69,12.92,68.38,10.18,NaN
105,105,Rawson,100,AR,1604282345,25,-43.30,-65.10,75.07,24.87,Hotel Punta León
178,178,Mount Gambier,15,AU,1604282602,36,-37.83,140.77,78.80,13.87,Commodore on the Park
230,230,Kidal,4,ML,1604282668,23,18.44,1.41,73.20,11.30,NaN
288,288,Port Lincoln,43,AU,1604282742,39,-34.73,135.87,77.25,13.47,Port Lincoln Hotel
338,338,Pirapozinho,0,BR,1604282734,46,-22.28,-51.50,71.60,11.41,Hotel Almanara
381,381,Jiménez,100,MX,1604282863,35,28.33,-105.40,66.18,12.39,NaN
423,423,Bariloche,0,AR,1604282761,42,-41.15,-71.31,64.40,11.41,Alma del Lago Suites & Spa
529,529,Kutum,0,SD,1604283050,34,14.20,24.67,69.85,16.82,NaN


### Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

In [52]:
# Setup template for adding hotel markers to the map
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Apply marker template to each row in vacation_df DataFrame
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]
locations = vacation_df[["Lat", "Lng"]]
# Add markers to heatmap
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))